# Assignment 1: Get Familiar with Python and Data Processing

Your Name: Samson Huang

Import necessary library tools into the environment; Please Google the name of each library to get feel for why they are all very useful.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
from sklearn import neighbors
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import (TimeSeriesSplit, KFold, ShuffleSplit,
                                     StratifiedKFold, GroupShuffleSplit,
                                     GroupKFold, StratifiedShuffleSplit)
from sklearn.model_selection import TimeSeriesSplit
from scipy.signal import hilbert, chirp 

sc = StandardScaler()

Step 1: Import the data from a accerelometer and gyroscope from "putty.log"

In [ ]:
#open the file
fin = open(r"C:\Users\sahua\Downloads\ENGR 492\Code\Assignment 1\putty.log","r+")

#store data in 2d array
data = [[]]

line = fin.readline()
while(line):
#   data.append(line.split('\n'))
   data.append(line.split(','))
   line = fin.readline()

data = data[1:]
# double check the data integrity
#print(data)

In [ ]:
#Data Visualization
aX = []
aY = []
aZ = []
bX = []
bY = []
bZ = []
t = []
I = []

time = 0
for log in data:
    aX.append(float(log[0]))
    aY.append(float(log[1])) ,
    aZ.append(float(log[2]))
    bX.append(float(log[6]))
    bY.append(float(log[7])) ,
    bZ.append(float(log[8]))
    t.append(time)
    I.append(float(log[10]))
    time += 1
     
    
plt.plot(t, aX)
plt.xlabel('time')
plt.ylabel("Acceleration X-axis")
plt.show()

plt.plot(t, aY)
plt.xlabel('time')
plt.ylabel("Acceleration Y-axis")
plt.show()

plt.plot(t, aZ)
plt.xlabel('time')
plt.ylabel("Acceleration Z-axis")
plt.show()

plt.plot(t, bX)
plt.xlabel('time')
plt.ylabel("Gyroscope X-axis")
plt.show()

plt.plot(t, bY)
plt.xlabel('time')
plt.ylabel("Gyroscope Y-axis")
plt.show()

plt.plot(t, bZ)
plt.xlabel('time')
plt.ylabel("Gyroscope Z-axis")
plt.show()


#print("TIme starts here",t)

Step 2: Clipping data 

In [ ]:
start = 100
amount = 15500
print(len(aZ))
data = data[start:amount]
print("length of the data",len(data))

Step 3: Chunking: chopping each X,Y,Z, acceleration and each X,Y,Z, angular rotation acceleration data into many many equal size windows (hint: 20 data points per chunk)

In [ ]:
import matplotlib.pyplot as plt

# Chunking acceleration for X,Y,Z
size = 20

acceleration = [data[i:i+size] for i in range(0, len(data), size)]

for chunk in acceleration:
    t_chunk = list(range(len(chunk)))
    aX_chunk = [float(log[0]) for log in chunk]
    aY_chunk = [float(log[1]) for log in chunk]
    aZ_chunk = [float(log[2]) for log in chunk]

    plt.plot(t_chunk, aX_chunk, label='Acceleration X-axis')
    plt.plot(t_chunk, aY_chunk, label='Acceleration Y-axis')
    plt.plot(t_chunk, aZ_chunk, label='Acceleration Z-axis')
    plt.xlabel('time')
    plt.ylabel('Acceleration')
    plt.legend()
    plt.show()
    
# Chunking angular rotation acceleration for X,Y,Z
angular = [data[i:i+size] for i in range(0, len(data), size)]

for chunk in angular:
    t_chunk = list(range(len(chunk)))
    bX_chunk = [float(log[6]) for log in chunk]
    bY_chunk = [float(log[7]) for log in chunk]
    bZ_chunk = [float(log[8]) for log in chunk]

    plt.plot(t_chunk, bX_chunk, label='Angular Rotation X-axis')
    plt.plot(t_chunk, bY_chunk, label='Angular Rotation Y-axis')
    plt.plot(t_chunk, bZ_chunk, label='Angular Rotation Z-axis')
    plt.xlabel('time')
    plt.ylabel('Angular Rotation Acceleration')
    plt.legend()
    plt.show()

Step 4: Feature Extraction: find (1) mean, (2) rms, (3)slope sign change, (4) number of positive peaks, (5) number of negative peaks, (6) number of zero crossing

In [ ]:
import numpy as np

def calculate(chunk):
    t_chunk = list(range(len(chunk)))
    aX_chunk = np.array([float(log[0]) for log in chunk])
    aY_chunk = np.array([float(log[1]) for log in chunk])
    aZ_chunk = np.array([float(log[2]) for log in chunk])

    # Mean
    mean_aX = np.mean(aX_chunk)
    mean_aY = np.mean(aY_chunk)
    mean_aZ = np.mean(aZ_chunk)

    # RMS
    rms_aX = np.sqrt(np.mean(aX_chunk**2))
    rms_aY = np.sqrt(np.mean(aY_chunk**2))
    rms_aZ = np.sqrt(np.mean(aZ_chunk**2))

    # Slope sign change
    sign_aX = np.sum(np.diff(np.sign(np.diff(aX_chunk))) != 0)
    sign_aY = np.sum(np.diff(np.sign(np.diff(aY_chunk))) != 0)
    sign_aZ = np.sum(np.diff(np.sign(np.diff(aZ_chunk))) != 0)

    # Positive peak
    pos_aX = len(np.where(np.diff(np.sign(np.diff(aX_chunk))) < 0)[0])
    pos_aY = len(np.where(np.diff(np.sign(np.diff(aY_chunk))) < 0)[0])
    pos_aZ = len(np.where(np.diff(np.sign(np.diff(aZ_chunk))) < 0)[0])

    # Negative peak
    neg_aX = len(np.where(np.diff(np.sign(np.diff(aX_chunk))) > 0)[0])
    neg_aY = len(np.where(np.diff(np.sign(np.diff(aY_chunk))) > 0)[0])
    neg_aZ = len(np.where(np.diff(np.sign(np.diff(aZ_chunk))) > 0)[0])

    # Zero crossing
    zero_aX = len(np.where(np.diff(np.sign(aX_chunk)))[0])
    zero_aY = len(np.where(np.diff(np.sign(aY_chunk)))[0])
    zero_aZ = len(np.where(np.diff(np.sign(aZ_chunk)))[0])

    return{
        'mean' : {'aX': mean_aX, 'aY': mean_aY, 'aZ': mean_aZ},
        'rms' : {'aX': rms_aX, 'aY': rms_aY, 'aZ': rms_aZ},
        'slope' : {'aX': sign_aX, 'aY': sign_aY, 'aZ': sign_aZ},
        'pos_peak' : {'aX': pos_aX, 'aY': pos_aY, 'aZ': pos_aZ},
        'neg_peak' : {'aX': neg_aX, 'aY': neg_aY, 'aZ': neg_aZ},
        'zero' : {'aX': zero_aX, 'aY': zero_aY, 'aZ': zero_aZ}
    }

for i, chunk in enumerate(acceleration):
    metrics = calculate(chunk)
    print("Frame #", i+1)
    print("Mean =", metrics['mean'])
    print("RMS =", metrics['rms'])
    print("Slope Sign Change =", metrics['slope'])
    print("Positive Peak =", metrics['pos_peak'])
    print("Negative Peak =", metrics['neg_peak'])
    print("Zero Crossings =", metrics['zero'])
    print()

Step 5: Reconstruction of the data matrix

In [ ]:
len(aX)

In [ ]:
def slope_sign_change(arr):
    """
    Checks if there is a sign change in the slope of the array.
    Returns True if there is a sign change, False otherwise.
    """
    sc = 0
    for i in range(len(arr)-2):
        slope1 = arr[i+1] - arr[i]
        slope2 = arr[i+2] - arr[i+1]
        if slope1 * slope2 < 0:
            sc += 1
            return
    return sc

In [ ]:
a = slope_sign_change([1, 2, -1, -2, 1])

In [ ]:
print(a)

In [ ]:
arr=np.array([1, 2, -1, -2, 1])

In [ ]:
sc = 0
for i in range(len(arr)-2):
    slope1 = arr[i+1] - arr[i]
    slope2 = arr[i+2] - arr[i+1]
    if slope1 * slope2 < 0:
        sc += 1
print (sc)